In [ ]:
import pathlib
import requests
import urllib
import boto3
import botocore

In [ ]:
SCHEME = "https"
NETLOC = "portal.opentopography.org"
PATH = "/API/otCatalog"

In [ ]:
params = {
        "productFormat": "PointCloud",
        "minx": 174.95818234951503,
        "miny": -41.00143843312103,
        "maxx": 175.17910282839554,
        "maxy": -40.836465206459664,
        "detail": False,
        "outputFormat": "json",
        "inlcude_federated": True
    }
data_info_stub = 'https://portal.opentopography.org/datasetMetadata?otCollectionID='

In [ ]:
data_url = urllib.parse.urlunparse((SCHEME, NETLOC, PATH, "", "", ""))

response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()

In [ ]:
json_response

In [ ]:
id_name=json_response['Datasets'][0]['Dataset']['alternateName']
id_name

# Try download file directly - get permission denied

In [ ]:
headers = {'User-Agent': USER_AGENT}
tile_url = "https://opentopography.s3.sdsc.edu/minio/pc-bulk/Wellington_2013/Wellington_2013_TileIndex.zip"
dest_path = r"C:\Users\pearsonra\Documents\data\Wakanae\local_cache\test.zip"
r = requests.get(tile_url, headers=headers, stream=True)
with open(dest_path, 'wb') as fd:
    for chunk in r.iter_content(): #chunk_size=chunk_size
        fd.write(chunk)

# AWS CLI that work
` C:\Users\pearsonra>aws s3 ls s3://pc-bulk/Wellington_2013/Wellington_2013_TileIndex.zip --recursive --endpoint-url https://opentopography.s3.sdsc.edu --no-sign-request`

`C:\Users\pearsonra>aws s3 cp s3://pc-bulk/Wellington_2013/Wellington_2013_TileIndex.zip . --recursive --endpoint-url https://opentopography.s3.sdsc.edu --no-sign-request`

# AWS keys

In [ ]:
aws_bucket = 'pc-bulk'
aws_endpoint_url = 'https://opentopography.s3.sdsc.edu'

tile_file_name = id_name + '_TileIndex.zip'


aws_secret_access_key = 'pc-bulk-3LoTa4'
ex_full_url = 'https://opentopography.s3.sdsc.edu/minio/pc-bulk/Wellington_2013'

# Boto client - success
Download the tile information

In [ ]:
client = boto3.client('s3',
                      endpoint_url=aws_endpoint_url,
                      config=botocore.config.Config(signature_version=botocore.UNSIGNED))

In [ ]:
metadata_key = id_name + '/' + id_name + '_TileIndex.zip'
client.download_file(aws_bucket, metadata_key, id_name + '_TileIndex.zip')

Example of getting all folders in OpenTopography LiDAR s3 storage

In [ ]:
paginator = client.get_paginator('list_objects')
result = paginator.paginate(Bucket=aws_bucket, Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

Example getting all objects in the Wellingto_2013 prefex

In [ ]:
paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=aws_bucket, Prefix=id_name)
number_of_files = 0
for page in pages:
    for obj in page['Contents']:
        number_of_files += 1
        #print(obj['Key'])
print(number_of_files)

Example printing more details about theWellington_2013 prefex - but only for the first page

In [ ]:
keys = []
if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None:
        print("No objects are available for %s" % aws_bucket)
    else:
        for obj in contents:
            keys.append(obj.get('Key'))
        print("There are %s objects available for %s\n--" % (len(keys), id_name))
        for k in keys:
            print(k)
else:
    print("There was an error with your request.")


# Boto code examples
Note even while specifying the aws_access_key_id and aws_secrete_access_key we still need to specify the full path to the buckets. 
### session

In [ ]:
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)



s3 = session.resource('s3', endpoint_url=aws_endpoint_url, 
                      config=botocore.config.Config(signature_version=botocore.UNSIGNED))

metadata_key = id_name + '/' + id_name + '_TileIndex.zip'
s3.Bucket(aws_bucket).download_file(metadata_key, 'test.zip')

### Client

In [ ]:
client = boto3.client('s3',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key,
                  endpoint_url=aws_endpoint_url,
                  config=botocore.config.Config(signature_version=botocore.UNSIGNED))

metadata_key = id_name + '/' + id_name + '_TileIndex.zip'
client.head_object(Bucket=aws_bucket, Key=metadata_key)

### Resource

In [ ]:
s3 = boto3.resource('s3', 
                    aws_access_key_id=aws_access_key_id, 
                    aws_secret_access_key=aws_secret_access_key,
                    endpoint_url=aws_endpoint_url,
                    config=botocore.config.Config(signature_version=botocore.UNSIGNED))

metadata_key = id_name + '/' + id_name + '_TileIndex.zip'
s3.Bucket(aws_bucket).download_file(metadata_key, 'test.zip')